In [13]:
!pip install bs_ds
!pip install fake_useragent
!pip install lxml
!pip install BeautifulSoup4

In [134]:
#Tipología y ciclo de vida de los datos
#Práctica 1
#Autor: Eduardo Bejar

#Importar lo necesario
import requests
import random
from bs4 import BeautifulSoup
import csv

#Función para remover etiquetas html
#Fuente: https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

#Definir header de archivo CSV
header = ['nombre', 'altura', 'puntaje', 'vistas', 'ubicación']

#Inicio de datos para CSV
data_csv = '['

#Fuente de datos elegida: SummitPost - https://www.summitpost.org/
#Sitio de información sobre elevaciones y montañas que existen en el mundo y unos pocos casos de fuera de la Tierra

#Contiene en total 629 páginas y 15.086 elevaciones que se consultan por separado 
#en páginas de hasta 24 elementos, a través de parámetro en la URL

#Número inicial de variable de recorrido de páginas
j = 603

#Loop de recorrido
while j < 630: #605:
    j += 1
    
    #URL de extracción de datos
    url = 'https://www.summitpost.org/object_list.php?search_in=name_only&map_1=1&order_type=DESC&object_type=1&orderby=elevation&page='
    
    #Agregar variable de recorrido de páginas
    indice = str(j)
    url_completa = url + indice
    #print(url_completa)

    #Técnica para cambiar de forma aleatoria de User Agent para hacer el web scraping
    #Fuente: https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/
    user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]
    for i in range(1,4):
        user_agent = random.choice(user_agent_list)

    headers = {'User-Agent': user_agent}

    #Enviar requerimiento a URL
    response = requests.get(url_completa,headers=headers, timeout=15)

    #Extracción de datos
    data = BeautifulSoup(response.text, 'html.parser')

    #Mediante revisión del código HTML de la página objetivo, se identifica que los
    #datos de interés están en DIVs
    divs=data.find('div');

    #Recorrer las DIVs buscando las clases que contienen los datos de interés
    for row in divs.findAll('div',{"class": "custom-card-item"}):
        
        #Nombre de la montaña
        nombre = row.findAll('p',{"class": "cci-title"})
        #Limpieza de los datos
        nombre_limpio = remove_html_tags(str(nombre))
        nombre_limpio = nombre_limpio.replace("\n", "")
        nombre_limpio = nombre_limpio.replace("[", "")
        nombre_limpio = nombre_limpio.replace("]", "")
        nombre_limpio = '"' + nombre_limpio.strip() + '"'
        print(nombre_limpio)

        #Altura de la montaña
        #Se identificaron casos en los cuales no está presente la etiqueta de altura
        #por lo que se les asigna valor de "ND" No Disponible para estos casos
        altura = row.findAll('span',{"class": "cci-elevation"}) or ("ND")

        #Limpieza de los datos
        if altura!="ND":
            altura_limpio = remove_html_tags(str(altura))
            partes_altura = altura_limpio.split(' ')
            altura_metros = partes_altura[5]
            altura_metros = altura_metros.replace("\n", "")
            altura_metros = altura_metros.replace("[", "")
            altura_metros = altura_metros.replace("]", "")
            altura = altura_metros
        print(altura)
        
        #Puntaje asignado por el sitio a la montaña
        #calculado en función de visitas y votos
        #Detalle sobre signficado del puntaje (score): https://www.summitpost.org/faq
        puntaje = row.findAll('span',{"class": "cci-score"})
        #Limpieza de los datos
        puntaje_limpio = remove_html_tags(str(puntaje))
        puntaje_limpio = puntaje_limpio.replace("\n", "")
        puntaje_limpio = puntaje_limpio.replace("[", "")
        puntaje_limpio = puntaje_limpio.replace("]", "")
        puntaje_limpio = puntaje_limpio.replace("Score: ", "")
        print(puntaje_limpio)
        
        #Cantidad de vistas o visitas a la página de la montaña
        vistas = row.findAll('span',{"class": "cci-hits"})
        #Limpieza de los datos
        vistas_limpio = remove_html_tags(str(vistas))
        vistas_limpio = vistas_limpio.replace("Views: ", "")
        vistas_limpio = vistas_limpio.replace("[", "")
        vistas_limpio = vistas_limpio.replace("]", "")
        print(vistas_limpio)
        
        lugar = row.findAll('p',{"class": "cci-location"})
        #Limpieza de los datos
        lugar_limpio = remove_html_tags(str(lugar))
        lugar_limpio = lugar_limpio.replace("\n", "")
        lugar_limpio = lugar_limpio.replace("[", "")
        lugar_limpio = lugar_limpio.replace("]", "")
        lugar_limpio = ''.join(lugar_limpio.split())
        lugar_limpio = '"' + lugar_limpio.strip() + '"'
        print(lugar_limpio)

        print("\n")
        print(j)
        print("\n")

        #Construcción de fila para esribir en el archivo CSV
        fila = '[' + nombre_limpio + ',' + altura + ',' + puntaje_limpio + ',' + vistas_limpio + ',' + lugar_limpio + '],'
        data_csv = data_csv + fila

data_csv = data_csv[:-1]
#Cierre de archivo CSV
data_csv = data_csv + ']'

print(data_csv)

with open('montanas.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data_csv)


print("Done")


"Mesquite Flat Dunes Highpoint"
30
88.61%
13739
"California,UnitedStates,NorthAmerica"


604


"Point Dume"
30
76.66%
14941
"California,UnitedStates,NorthAmerica"


604


"Cae Du"
30
88.19%
7809
"Snowdonia,Wales,Europe"


604


"Volcán de Lodo El Totumo"
30
86.37%
6629
"Colombia,SouthAmerica"


604


"Easter Egg Rocks (5.2-5.9)"
27
71.06%
1222
"Maryland,UnitedStates,NorthAmerica"


604


"Jam Box (5.0-5.10+)"
26
74.01%
1219
"Maryland,UnitedStates,NorthAmerica"


604


"Birthday Crag"
25
71.06%
2307
"RasAlKhaimah,UnitedArabEmirates,Asia"


604


"St. Non's Bay"
25
87.76%
5519
"Wales,Europe"


604


"Nasenlöcherturm / Nostrils Tower 5.5 - 5.11d"
22
81.84%
3467
"Bavaria,Germany,Europe"


604


"Rockall"
21
84.81%
8308
"Europe"


604


"Kato Zakros crag, east Crete"
20
71.06%
265
"Greece,Europe"


604


"Katsounaki gorge, east Crete"
20
0%
321
"Greece,Europe"


604


"Amatou gorge, east Crete"
20
0%
256
"Greece,Europe"


604


"La Roque"
20
81.42%
5417
"Normandy,France,Europe"


604


"Por

"Unleashed Wall, 5.7-5.12b"
ND
74.01%
801
"Texas,UnitedStates,NorthAmerica"


609


"Iron Maiden Corridor, 5.8-5.12a"
ND
74.01%
202
"California,UnitedStates,NorthAmerica"


609


"East End, 5.6-5.10c"
ND
83.1%
6236
"Alberta,Canada,NorthAmerica"


609


"Stone Wall, 5.10b-5.11b"
ND
82.48%
3941
"Nevada,UnitedStates,NorthAmerica"


609


"Bumblie Wall, 5.6-5.11c"
ND
73.06%
579
"Idaho,UnitedStates,NorthAmerica"


609


"Mount Spry"
ND
87.76%
5971
"Utah,UnitedStates,NorthAmerica"


609


"Byrd Pinnacle, 5.7-5.10c"
ND
80.49%
2666
"Nevada,UnitedStates,NorthAmerica"


609


"Bridge Mountain, 5.10-5.11+"
ND
80.49%
5863
"Utah,UnitedStates,NorthAmerica"


609


"Spider Wall, 5.5-5.9"
ND
73.06%
764
"California,UnitedStates,NorthAmerica"


609


"Ostrzyca"
ND
76.46%
5027
"Poland,Europe"


609


"Sundance Peak (UT)"
ND
74.01%
6355
"Utah,UnitedStates,NorthAmerica"


609


"Isles Corridor, 5.6-5.12b"
ND
74.92%
580
"California,UnitedStates,NorthAmerica"


609


"Left Entrance Wall, 5.10a-5.11b"
ND
73.0

"Mother's Day Buttress, 5.7-5.10c"
ND
85.87%
9466
"Alberta,Canada,NorthAmerica"


614


"Dutzi Rock, 5.6-5.11b"
ND
72.08%
427
"California,UnitedStates,NorthAmerica"


614


"Lynch Crag, 5.9-5.11d"
ND
75.81%
1308
"California,UnitedStates,"


614


"Serpent's Wall (5.7-5.11b)"
ND
76.66%
2125
"Texas,UnitedStates,NorthAmerica"


614


"Clear Creek Mountain"
ND
77.48%
5141
"Utah,UnitedStates,NorthAmerica"


614


"Castle Rock, North Face, 5.0-5.12a"
ND
73.06%
597
"California,UnitedStates,NorthAmerica"


614


"Donohue Peak from Kuna Creek Basin"
ND
71.06%
2051
"California,YosemiteNationalPark,UnitedStates,"


614


"Out of Control Area, 5.9-5.10c"
ND
74.63%
1613
"Nevada,UnitedStates,NorthAmerica"


614


"Star Wars Rock, 5.9-5.12c"
ND
74.01%
449
"California,UnitedStates,NorthAmerica"


614


"Ghezghoonchal"
ND
73.81%
874
"Tehran,Iran,Asia"


614


"Gypsy Wall (5.7-5.10d)"
ND
77.48%
1698
"Texas,UnitedStates,NorthAmerica"


614


"Chalkboard Dome, 5.5-5.11"
ND
0%
269
"Arizona,UnitedStates,Nor

"Brimham Rocks"
ND
77.48%
2971
"England,Europe"


619


"Rocky Peak"
ND
82.48%
8825
"Utah,UnitedStates,NorthAmerica"


619


"Tunnel Wall, 5.10-5.13+"
ND
84.82%
4778
"Utah,UnitedStates,NorthAmerica"


619


"Dead Cats Wall (5.5-5.12a)"
ND
79.04%
4095
"Texas,UnitedStates,NorthAmerica"


619


"House of Cards, Left, 5.8-5.10d"
ND
74.01%
3544
"Alberta,Canada,NorthAmerica"


619


"Cold Springs Dome"
ND
85.36%
6790
"California,UnitedStates,NorthAmerica"


619


"Asian Fever Buttress, 5.4-5.7"
ND
74.92%
1493
"California,UnitedStates,NorthAmerica"


619


"Walt’s Rocks, 5.8-5.13d"
ND
79.78%
1279
"California,UnitedStates,NorthAmerica"


619


"Super Slab, 5.7-5.10c"
ND
72.08%
329
"California,UnitedStates,NorthAmerica"


619


"Orient Bay"
ND
72.08%
9729
"ontario,Canada,NorthAmerica"


619


"Mounds, 5.8-5.10d"
ND
78.27%
1569
"California,UnitedStates,NorthAmerica"


619


"Mission Gorge, San Diego's Local Crag"
ND
85.36%
43662
"California,UnitedStates,NorthAmerica"


619


"Alberta Jam, 5.9-5.

"Paroi de la Vierge"
ND
85.36%
1435
"Provence-Côted'Azur,France,Europe"


624


"Peewee Rock, 5.4-5.10d"
ND
72.08%
383
"California,UnitedStates,NorthAmerica"


624


"Castle Rock (Shock) Buttress, 5.6-5.12c"
ND
72.08%
1297
"Idaho,UnitedStates,NorthAmerica"


624


"Middle Earth Wall (5.6-5.11a)"
ND
76.66%
2398
"Texas,UnitedStates,NorthAmerica"


624


""Stansbury Island South""
ND
85.36%
8815
"Utah,UnitedStates,NorthAmerica"


624


"Curry’s, 5.8-5.13b"
ND
74.01%
726
"Wyoming,UnitedStates,NorthAmerica"


624


"Molar Tower"
ND
74.01%
4345
"Utah,UnitedStates,NorthAmerica"


624


"Ixtlan Buttress, 5.9-5.12a"
ND
82.48%
4251
"Nevada,UnitedStates,NorthAmerica"


624


"Sekrit Hill "Point 1040""
ND
69.64%
384
""


624


"Taylor's Falls"
ND
74.53%
15686
"Minnesota,UnitedStates,NorthAmerica"


624


"Gold Ridge"
ND
76.66%
6689
"Utah,UnitedStates,NorthAmerica"


624


"The Taco, 5.7-5.11a"
ND
72.08%
607
"Idaho,UnitedStates,NorthAmerica"


624


"Aeolian Wall, 5.10a-5.11d"
ND
81.18%
5584
"Nevad

"Middle East Wall"
ND
74.92%
9239
"Washington,UnitedStates,NorthAmerica"


629


"Jimmy Cliff, 5.4-5.12b"
ND
77.48%
1257
"California,UnitedStates,NorthAmerica"


629


"Pala delle Lastiele"
ND
81.18%
1570
"ValledelSarca,Trento,Italy,Europe"


629


"Conan’s Corridor, 5.8-5.10d"
ND
81.18%
2773
"California,UnitedStates,NorthAmerica"


629


"Fairfield Hill West"
ND
77.48%
1082
"Wyoming,UnitedStates,NorthAmerica"


629


"Building Blocks, 5.5-5.12c"
ND
72.08%
672
"Idaho,UnitedStates,NorthAmerica"


629


"Mount Grossvogel, 5.5-5.11a"
ND
71.06%
267
"California,UnitedStates,NorthAmerica"


629


"Devil's Thumb, Alaska"
ND
0%
2520
"Alaska,UnitedStates,NorthAmerica"


629


"North Six Shooter"
ND
81.52%
12945
"Utah,UnitedStates,NorthAmerica"


629


"Mont Avril"
ND
80.49%
1494
"AostaValley,Suisse,Europe"


629


"Upper and Lower Creekside, 5.5-5.12b"
ND
77.48%
1470
"Idaho,UnitedStates,NorthAmerica"


629


"Checkered Demon, 5.8-5.13c"
ND
74.01%
646
"Idaho,UnitedStates,NorthAmerica"


629


"T